In [1]:
%load_ext autoreload
%autoreload 2

from pandas import read_excel
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score, KFold, train_test_split, StratifiedKFold, cross_validate
from sklearn.metrics import confusion_matrix, mean_squared_error, classification_report, f1_score, mean_squared_log_error, recall_score, accuracy_score
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, RandomForestRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn import svm, tree 
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import EditedNearestNeighbours, ClusterCentroids, RandomUnderSampler 
from collections import Counter

import sys
sys.path.insert(0, '/Users/pauor506/Documents/PP_active_learning/')
import functions as fun

In [3]:
# Reading dataset (from https://www.kaggle.com/datasets/mathchi/diabetes-data-set?resource=download)
diabetes = pd.read_csv('/Users/pauor506/Documents/PP_active_learning/BasicClassification_easydataset/diabetes.csv')

diabetes['Glucose']=diabetes['Glucose'].replace(0,diabetes['Glucose'].mean())
diabetes['BloodPressure']=diabetes['BloodPressure'].replace(0,diabetes['BloodPressure'].mean())
diabetes['SkinThickness']=diabetes['SkinThickness'].replace(0,diabetes['SkinThickness'].mean())
diabetes['Insulin']=diabetes['Insulin'].replace(0,diabetes['Insulin'].mean())
diabetes['BMI']=diabetes['BMI'].replace(0,diabetes['BMI'].mean())

# Creating X and y sets
X_init = diabetes.drop(['Outcome'], axis=1).to_numpy() # I am using all the features for prediction
X = StandardScaler().fit_transform(X_init) # Scale the data 
y = diabetes['Outcome'].to_numpy()
# Balancing dataset
X, y = SMOTE(random_state=8472352).fit_resample(X, y)

# Sets for training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6752)


In [4]:
# Models
knn = KNeighborsClassifier(n_neighbors = 5)
svmm = svm.SVC(random_state=0)
ada_clf = AdaBoostClassifier(random_state=0) #DecisionTreeClassifier initialized with max_depth=1 and 50 trees

models = [knn, svmm, ada_clf]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

sspace_knn = {'n_neighbors':[i for i in np.arange(5,10)], 'leaf_size':[i for i in np.arange(20,50,5)]}
sspace_svmm = {'kernel': ["poly", "rbf", "sigmoid"], 'gamma': [1.0, 0.1, 1e-2], 'C': [1, 10, 30, 50, 80, 100]}
sspace_ada = {'n_estimators': [10,30,50,70,100], 'learning_rate': [1.5, 1.0, 0.1, 0.01]}

rnd_knn = RandomizedSearchCV(estimator=knn, param_grid=sspace_knn, scoring='accuracy', 
                       cv=10, return_train_score=True, random_state=0, n_jobs=-1)
rnd_svmm = RandomizedSearchCV(estimator=svmm, param_grid=sspace_knn, scoring='accuracy', 
                       cv=10, return_train_score=True, random_state=0, n_jobs=-1)
rnd_ada = RandomizedSearchCV(estimator=ada_clf, param_grid=sspace_knn, scoring='accuracy', 
                       cv=10, return_train_score=True, random_state=0, n_jobs=-1)

In [ ]:
#Training and Prediction
result_knn = rnd_knn.fit(X_train, y_train)
resukt_svmm = rnd_svmm